In [20]:
import numpy as np
import pandas as pd
from collections import defaultdict

### Globals

In [3]:
OUTPUT_PATH = 'Outputs/'
FEATURES_PATH = 'Features/'
ALGORITHMS = ['dp1salida', 'dp2salida', 'greedyoutput']
FEATURES = ['num_elem',
            'cap',
            'cap_mean_w',
            'cap_median_w', 
            'cap_desv_w', 
            'mean_w_mean_v',
            'median_w_median_v', 
            'desv_w_desv_v',
            'max_w_min_w',
            'max_v_min_v',
            'mean_w',
            'median_w', 
            'desv_w', 
            'min_w', 
            'max_w', 
            'mean_v', 
            'median_v', 
            'desv_v', 
            'min_v', 
            'max_v',
            'p_coef' 
           ]

### Helpers

In [4]:
def get_features(id_):
    #return [eval(x) if not x.isalpha() else x for x in (FEATURES_PATH + id_,'r').read().split()]
    return [100,5,2,1,3,3,6,4,3,1,1,3,4,4,5,6,6,6,2,1,-1]

In [15]:
alg_data = defaultdict(list)

for alg in ALGORITHMS:
    file = open(OUTPUT_PATH + alg,'r')
    alg_data[alg] = defaultdict(list)
    for row in file:
        try: # para cuando hay
            id_, fo, time = row.split()
            alg_data[alg][id_].append([(int)(fo), (float)(time)])
        except:
            pass

In [16]:
instances = alg_data['dp1salida'].keys()
train = []
test = [-1] * len(instances)*3600
count = 0
for inst in instances:
    fts = get_features(inst)
    for t in range(10):
        train.append([inst, *fts, t+1])
        best_fo = 0
        for alg in ALGORITHMS:
            try:
                local_best = max([i for i in alg_data[alg][inst] if i[1] <= t+1])[0] # lista de valores menores al tiempo
                if local_best > best_fo:
                    best_fo = local_best
                    test[count] = alg
            except:
                pass
        count += 1
                

In [17]:
data = pd.DataFrame(data=train, columns=['id', *FEATURES, 't'])

In [19]:
data

,id,num_elem,cap,cap_mean_w,cap_median_w,cap_desv_w,mean_w_mean_v,median_w_median_v,desv_w_desv_v,max_w_min_w,...,desv_w,min_w,max_w,mean_v,median_v,desv_v,min_v,max_v,p_coef,t
0,20629,100,5,2,1,3,3,6,4,3,...,4,4,5,6,6,6,2,1,-10,1
1,20629,100,5,2,1,3,3,6,4,3,...,4,4,5,6,6,6,2,1,-10,2
2,20629,100,5,2,1,3,3,6,4,3,...,4,4,5,6,6,6,2,1,-10,3
3,20629,100,5,2,1,3,3,6,4,3,...,4,4,5,6,6,6,2,1,-10,4
4,20629,100,5,2,1,3,3,6,4,3,...,4,4,5,6,6,6,2,1,-10,5
5,20629,100,5,2,1,3,3,6,4,3,...,4,4,5,6,6,6,2,1,-10,6
6,20629,100,5,2,1,3,3,6,4,3,...,4,4,5,6,6,6,2,1,-10,7
7,20629,100,5,2,1,3,3,6,4,3,...,4,4,5,6,6,6,2,1,-10,8
8,20629,100,5,2,1,3,3,6,4,3,...,4,4,5,6,6,6,2,1,-10,9
9,20629,100,5,2,1,3,3,6,4,3,...,4,4,5,6,6,6,2,1,-10,10
